# ontology

> RDF ontology loading and meta-graph navigation for RLM

In [ ]:
#| default_exp ontology

## Overview

This module implements Stages 1-2 of the trajectory: Define the Ontology "Context Model" and provide bounded view primitives for progressive disclosure.

**Stage 1**: Meta-graph scaffolding with navigation indexes  
**Stage 2**: Bounded view primitives for safe graph exploration

### Design Principles

- **Handles, not dumps**: Return graph handles with bounded view operations
- **Meta-graph scaffolding**: Build navigation indexes (labels, hierarchy, properties)
- **Progressive disclosure**: Small summaries guide exploration
- **RLM-compatible**: Works with namespace-explicit `rlm_run()`

### Context Model

From the trajectory document:
> The *root model never gets a graph dump*. It gets a handle name (e.g. `ont`, `res_0`) and uses bounded view operations.

## Imports

In [ ]:
#| export
from rdflib import Graph, Namespace, RDF, RDFS, OWL, URIRef, Literal, SKOS, DCTERMS
from pathlib import Path
from collections import Counter, defaultdict
from dataclasses import dataclass, field
from fastcore.basics import AttrDict

# Additional namespaces for ontology metadata (Widoco guide)
VANN = Namespace('http://purl.org/vocab/vann/')
DC = Namespace('http://purl.org/dc/elements/1.1/')

## Graph Loading

In [ ]:
#| export
def load_ontology(path: str | Path, ns: dict, name: str = 'ont') -> str:
    """Load an RDF ontology file into namespace as a Graph handle.
    
    Args:
        path: Path to ontology file (.ttl, .rdf, .owl)
        ns: Namespace dict where Graph will be stored
        name: Variable name for the Graph handle
        
    Returns:
        Summary string describing what was loaded
    """
    g = Graph()
    g.parse(path)
    ns[name] = g
    
    return f"Loaded {len(g)} triples from {Path(path).name} into '{name}'"

In [ ]:
# Test loading prov.ttl
test_ns = {}
result = load_ontology('ontology/prov.ttl', test_ns, name='prov_ont')
print(result)
assert 'prov_ont' in test_ns
assert isinstance(test_ns['prov_ont'], Graph)
assert len(test_ns['prov_ont']) > 0
print(f"✓ Loaded {len(test_ns['prov_ont'])} triples")

## Meta-Graph Navigation

Build navigation scaffolding from a Graph to enable progressive disclosure.
This is what goes in the REPL environment, not the graph itself.

In [ ]:
#| export
@dataclass
class GraphMeta:
    """Meta-graph navigation scaffolding for an RDF Graph.
    
    This is REPL-resident and provides bounded views over the graph.
    Indexes discovered in dialogs/inspect_tools.ipynb exploration.
    """
    graph: Graph
    name: str = 'ont'
    
    # Computed lazily
    _namespaces: dict = field(default=None, init=False, repr=False)
    _classes: list = field(default=None, init=False, repr=False)
    _properties: list = field(default=None, init=False, repr=False)
    _individuals: list = field(default=None, init=False, repr=False)
    _labels: dict = field(default=None, init=False, repr=False)
    _by_label: dict = field(default=None, init=False, repr=False)
    _subs: dict = field(default=None, init=False, repr=False)
    _supers: dict = field(default=None, init=False, repr=False)
    _doms: dict = field(default=None, init=False, repr=False)
    _rngs: dict = field(default=None, init=False, repr=False)
    _pred_freq: Counter = field(default=None, init=False, repr=False)
    
    @property
    def triple_count(self) -> int:
        """Total number of triples in graph."""
        return len(self.graph)
    
    @property
    def namespaces(self) -> dict:
        """Get namespace prefix bindings."""
        if self._namespaces is None:
            self._namespaces = {prefix: str(ns) for prefix, ns in self.graph.namespaces()}
        return self._namespaces
    
    @property
    def classes(self) -> list:
        """Get all OWL/RDFS classes (URIs only, sorted)."""
        if self._classes is None:
            classes = set(
                self.graph.subjects(RDF.type, OWL.Class)
            ).union(
                self.graph.subjects(RDF.type, RDFS.Class)
            )
            self._classes = sorted([str(c) for c in classes])
        return self._classes
    
    @property
    def properties(self) -> list:
        """Get all properties (URIs only, sorted)."""
        if self._properties is None:
            props = set(
                self.graph.subjects(RDF.type, OWL.ObjectProperty)
            ).union(
                self.graph.subjects(RDF.type, OWL.DatatypeProperty)
            ).union(
                self.graph.subjects(RDF.type, OWL.AnnotationProperty)
            ).union(
                self.graph.subjects(RDF.type, RDF.Property)
            )
            self._properties = sorted([str(p) for p in props])
        return self._properties
    
    @property
    def individuals(self) -> list:
        """Get all named individuals (URIs only, sorted)."""
        if self._individuals is None:
            inds = set(self.graph.subjects(RDF.type, OWL.NamedIndividual))
            self._individuals = sorted([str(i) for i in inds])
        return self._individuals
    
    @property
    def labels(self) -> dict:
        """Get label index: URI -> label string."""
        if self._labels is None:
            self._labels = {}
            for s, o in self.graph.subject_objects(RDFS.label):
                self._labels[str(s)] = str(o)
        return self._labels
    
    @property
    def by_label(self) -> dict:
        """Get inverted label index: label_text -> list of URIs."""
        if self._by_label is None:
            inv = defaultdict(list)
            for uri, lbl in self.labels.items():
                inv[lbl.lower()].append(uri)
            self._by_label = dict(inv)
        return self._by_label
    
    @property
    def subs(self) -> dict:
        """Get subclass relationships: superclass_uri -> list of subclass_uris."""
        if self._subs is None:
            subs_dict = defaultdict(list)
            for s, _, o in self.graph.triples((None, RDFS.subClassOf, None)):
                if isinstance(o, URIRef):
                    subs_dict[str(o)].append(str(s))
            self._subs = dict(subs_dict)
        return self._subs
    
    @property
    def supers(self) -> dict:
        """Get superclass relationships: subclass_uri -> list of superclass_uris."""
        if self._supers is None:
            supers_dict = defaultdict(list)
            for s, _, o in self.graph.triples((None, RDFS.subClassOf, None)):
                if isinstance(o, URIRef):
                    supers_dict[str(s)].append(str(o))
            self._supers = dict(supers_dict)
        return self._supers
    
    @property
    def doms(self) -> dict:
        """Get property domains: property_uri -> domain_uri."""
        if self._doms is None:
            self._doms = {str(s): str(o) for s, _, o in self.graph.triples((None, RDFS.domain, None))}
        return self._doms
    
    @property
    def rngs(self) -> dict:
        """Get property ranges: property_uri -> range_uri."""
        if self._rngs is None:
            self._rngs = {str(s): str(o) for s, _, o in self.graph.triples((None, RDFS.range, None))}
        return self._rngs
    
    @property
    def pred_freq(self) -> Counter:
        """Get predicate frequency counts (cached)."""
        if self._pred_freq is None:
            from collections import Counter
            self._pred_freq = Counter(str(p) for s, p, o in self.graph.triples((None, None, None)))
        return self._pred_freq
    
    def summary(self) -> str:
        """Generate a summary of the graph for display."""
        lines = [
            f"Graph '{self.name}': {self.triple_count:,} triples",
            f"Classes: {len(self.classes)}",
            f"Properties: {len(self.properties)}",
            f"Individuals: {len(self.individuals)}",
            f"Namespaces: {', '.join(self.namespaces.keys())}"
        ]
        return '\n'.join(lines)

In [ ]:
# Test GraphMeta with prov ontology
prov_g = test_ns['prov_ont']
meta = GraphMeta(prov_g, name='prov')

print(meta.summary())
print()
print(f"Sample classes (first 5): {meta.classes[:5]}")
print(f"Sample properties (first 5): {meta.properties[:5]}")
print(f"Namespaces: {list(meta.namespaces.keys())}")

## Bounded View Functions (Stage 1)

Basic operations on GraphMeta that return small, bounded summaries:

- **graph_stats()**: Overall graph statistics
- **search_by_label()**: Simple label-based search
- **describe_entity()**: Get entity description with sample triples

These provide the foundation for progressive disclosure.

In [ ]:
#| export
def graph_stats(meta: GraphMeta) -> str:
    """Get graph statistics summary."""
    return meta.summary()

In [ ]:
#| export
def search_entity(meta: GraphMeta, query: str, limit: int = 10,
                  search_in: str = 'all') -> list:
    """Search for entities by label, IRI, or localname.

    Args:
        meta: GraphMeta to search
        query: Search string (case-insensitive substring match)
        limit: Maximum results to return
        search_in: Where to search - 'label', 'iri', 'localname', or 'all'

    Returns:
        List of dicts: [{'uri': str, 'label': str, 'match_type': str}, ...]
    """
    query_lower = query.lower()
    matches = []

    # Search in labels
    if search_in in ('label', 'all'):
        for uri, label in meta.labels.items():
            if query_lower in label.lower():
                matches.append({
                    'uri': uri,
                    'label': label,
                    'match_type': 'label'
                })

    # Search in full IRIs
    if search_in in ('iri', 'all'):
        all_uris = set(meta.classes + meta.properties + meta.individuals)
        for uri in all_uris:
            if query_lower in uri.lower() and not any(m['uri'] == uri for m in matches):
                label = meta.labels.get(uri, uri)
                matches.append({
                    'uri': uri,
                    'label': label,
                    'match_type': 'iri'
                })

    # Search in localnames (fragment or last path segment)
    if search_in in ('localname', 'all'):
        all_uris = set(meta.classes + meta.properties + meta.individuals)
        for uri in all_uris:
            # Extract localname (after # or last /)
            if '#' in uri:
                localname = uri.split('#')[-1]
            else:
                localname = uri.split('/')[-1]

            if query_lower in localname.lower() and not any(m['uri'] == uri for m in matches):
                label = meta.labels.get(uri, uri)
                matches.append({
                    'uri': uri,
                    'label': label,
                    'match_type': 'localname'
                })

    return matches[:limit]


def search_by_label(meta: GraphMeta, search: str, limit: int = 10) -> list:
    """Search for entities by label substring (case-insensitive).

    Backward-compatible wrapper around search_entity().

    Args:
        meta: GraphMeta to search
        search: Substring to search for in labels
        limit: Maximum results to return

    Returns:
        List of (URI, label) tuples
    """
    results = search_entity(meta, search, limit=limit, search_in='label')
    return [(r['uri'], r['label']) for r in results]

In [ ]:
# Test search_entity
results = search_entity(meta, 'activity', limit=5)
print(f"Found {len(results)} matches for 'activity':")
for r in results:
    print(f"  {r['label']}: {r['uri']} ({r['match_type']})")

# Test different search modes
print("\nSearch by IRI only:")
iri_results = search_entity(meta, 'prov', search_in='iri', limit=3)
for r in iri_results:
    print(f"  {r['label']}: {r['uri']}")

# Test backward compatibility
print("\nBackward compatibility test:")
legacy_results = search_by_label(meta, 'activity', limit=5)
print(f"Found {len(legacy_results)} matches using search_by_label():")
for uri, label in legacy_results:
    print(f"  {label}: {uri}")

In [ ]:
#| export
def describe_entity(meta: GraphMeta, uri: str, limit: int = 20) -> dict:
    """Get bounded description of an entity.
    
    Args:
        meta: GraphMeta containing the entity
        uri: URI of entity to describe
        limit: Max number of triples to include
        
    Returns:
        Dict with label, types, and sample triples
    """
    from rdflib import URIRef
    
    entity = URIRef(uri)
    
    # Get label
    label = meta.labels.get(uri, uri)
    
    # Get types
    types = [str(t) for t in meta.graph.objects(entity, RDF.type)]
    
    # Get sample of outgoing triples
    outgoing = []
    for p, o in list(meta.graph.predicate_objects(entity))[:limit]:
        outgoing.append((str(p), str(o)))
    
    # Get comment if available
    comments = list(meta.graph.objects(entity, RDFS.comment))
    comment = str(comments[0]) if comments else None
    
    return {
        'uri': uri,
        'label': label,
        'types': types,
        'comment': comment,
        'outgoing_sample': outgoing[:limit]  # FIX: Use limit parameter, not hardcoded [:10]
    }

In [ ]:
# Test describe_entity
# Find the Activity class
activity_uri = 'http://www.w3.org/ns/prov#Activity'
desc = describe_entity(meta, activity_uri)

print(f"Label: {desc['label']}")
print(f"Types: {desc['types']}")
print(f"Comment: {desc['comment'][:100]}..." if desc['comment'] else "No comment")
print(f"Outgoing triples: {len(desc['outgoing_sample'])}")

### Stage 2: Progressive Disclosure Primitives

Advanced bounded view operations that enable root models to explore graphs iteratively:

- **search_entity()**: Multi-mode entity search (label/IRI/localname)
- **probe_relationships()**: One-hop neighbor exploration with filtering
- **find_path()**: BFS path finding between entities
- **predicate_frequency()**: Usage analysis for understanding graph structure

These primitives answer questions like:
- "Is X defined?" → `search_entity()`
- "What connects A to B?" → `find_path()`
- "What are the most important predicates?" → `predicate_frequency()`
- "What does X relate to?" → `probe_relationships()`

In [ ]:
#| export
def probe_relationships(meta: GraphMeta, uri: str, predicate: str = None,
                        direction: str = 'both', limit: int = 20) -> dict:
    """Get one-hop neighbors of an entity, optionally filtered by predicate.

    Args:
        meta: GraphMeta containing the entity
        uri: URI of entity to probe
        predicate: Optional predicate URI to filter by
        direction: 'out', 'in', or 'both' (default: 'both')
        limit: Maximum neighbors to return per direction

    Returns:
        {
            'uri': str, 'label': str,
            'outgoing': [{'predicate': str, 'pred_label': str,
                          'object': str, 'obj_label': str}, ...],
            'incoming': [{'subject': str, 'subj_label': str,
                          'predicate': str, 'pred_label': str}, ...],
            'outgoing_count': int, 'incoming_count': int
        }
    """
    from rdflib import URIRef

    entity = URIRef(uri)
    entity_label = meta.labels.get(uri, uri)

    outgoing = []
    incoming = []

    # Get outgoing triples (entity as subject)
    if direction in ('out', 'both'):
        pred_filter = URIRef(predicate) if predicate else None
        triples = list(meta.graph.triples((entity, pred_filter, None)))

        for s, p, o in triples[:limit]:
            pred_uri = str(p)
            obj_uri = str(o)
            outgoing.append({
                'predicate': pred_uri,
                'pred_label': meta.labels.get(pred_uri, pred_uri),
                'object': obj_uri,
                'obj_label': meta.labels.get(obj_uri, obj_uri)
            })

    # Get incoming triples (entity as object)
    if direction in ('in', 'both'):
        pred_filter = URIRef(predicate) if predicate else None
        triples = list(meta.graph.triples((None, pred_filter, entity)))

        for s, p, o in triples[:limit]:
            subj_uri = str(s)
            pred_uri = str(p)
            incoming.append({
                'subject': subj_uri,
                'subj_label': meta.labels.get(subj_uri, subj_uri),
                'predicate': pred_uri,
                'pred_label': meta.labels.get(pred_uri, pred_uri)
            })

    # Count total (not just limited sample)
    pred_filter = URIRef(predicate) if predicate else None
    if direction in ('out', 'both'):
        outgoing_count = sum(1 for _ in meta.graph.triples((entity, pred_filter, None)))
    else:
        outgoing_count = 0

    if direction in ('in', 'both'):
        incoming_count = sum(1 for _ in meta.graph.triples((None, pred_filter, entity)))
    else:
        incoming_count = 0

    return {
        'uri': uri,
        'label': entity_label,
        'outgoing': outgoing,
        'incoming': incoming,
        'outgoing_count': outgoing_count,
        'incoming_count': incoming_count
    }

In [ ]:
#| export
def find_path(meta: GraphMeta, source: str, target: str,
              max_depth: int = 2, limit: int = 10) -> list:
    """Find predicates connecting two entities using BFS.

    Answers "What predicates connect A to B?"

    Args:
        meta: GraphMeta to search
        source: Source entity URI
        target: Target entity URI
        max_depth: Maximum path length (default: 2)
        limit: Maximum paths to return

    Returns:
        List of paths, each path is list of steps:
        [{'from': uri, 'predicate': uri, 'to': uri, 'direction': 'out'|'in'}, ...]
    """
    from rdflib import URIRef
    from collections import deque

    source_uri = URIRef(source)
    target_uri = URIRef(target)

    # BFS to find paths
    queue = deque([(source_uri, [])])  # (current_node, path_so_far)
    visited = set()
    paths_found = []

    while queue and len(paths_found) < limit:
        current, path = queue.popleft()

        # Skip if we've exceeded max depth
        if len(path) >= max_depth:
            continue

        # Skip if visited (but allow revisiting in different paths up to limit)
        path_key = (current, tuple(step['predicate'] for step in path))
        if path_key in visited:
            continue
        visited.add(path_key)

        # Check if we reached the target
        if current == target_uri:
            paths_found.append(path)
            continue

        # Explore outgoing edges
        for s, p, o in meta.graph.triples((current, None, None)):
            if isinstance(o, URIRef):
                step = {
                    'from': str(s),
                    'predicate': str(p),
                    'to': str(o),
                    'direction': 'out'
                }
                queue.append((o, path + [step]))

        # Explore incoming edges
        for s, p, o in meta.graph.triples((None, None, current)):
            if isinstance(s, URIRef):
                step = {
                    'from': str(current),
                    'predicate': str(p),
                    'to': str(s),
                    'direction': 'in'
                }
                queue.append((s, path + [step]))

    return paths_found

In [ ]:
#| export
def predicate_frequency(meta: GraphMeta, limit: int = 20,
                        predicate_type: str = None) -> list:
    """Get predicates ranked by frequency of use.

    Args:
        meta: GraphMeta to analyze
        limit: Maximum predicates to return
        predicate_type: Optional filter - 'object', 'datatype', 'annotation'

    Returns:
        List of dicts: [{'predicate': str, 'label': str, 'count': int,
                         'sample_subject': str, 'sample_object': str}, ...]
    """
    from rdflib import URIRef
    from collections import Counter

    # Get frequency counter (cached)
    freq = meta.pred_freq

    # Filter by type if requested
    if predicate_type:
        type_props = set()

        if predicate_type == 'object':
            type_props = set(meta.graph.subjects(RDF.type, OWL.ObjectProperty))
        elif predicate_type == 'datatype':
            type_props = set(meta.graph.subjects(RDF.type, OWL.DatatypeProperty))
        elif predicate_type == 'annotation':
            type_props = set(meta.graph.subjects(RDF.type, OWL.AnnotationProperty))

        # Filter frequency counts to only include predicates of this type
        # FIX: Wrap in Counter so most_common() works
        filtered_freq = Counter({str(p): count for p, count in freq.items() if URIRef(p) in type_props})
    else:
        filtered_freq = freq

    # Get top N by frequency
    top_predicates = filtered_freq.most_common(limit)

    # Build result list with samples
    results = []
    for pred_uri, count in top_predicates:
        # Get label
        pred_label = meta.labels.get(pred_uri, pred_uri)

        # Get sample triple
        sample_triple = next(meta.graph.triples((None, URIRef(pred_uri), None)), None)

        if sample_triple:
            sample_subj = str(sample_triple[0])
            sample_obj = str(sample_triple[2])
        else:
            sample_subj = None
            sample_obj = None

        results.append({
            'predicate': pred_uri,
            'label': pred_label,
            'count': count,
            'sample_subject': sample_subj,
            'sample_object': sample_obj
        })

    return results

In [ ]:
# Test probe_relationships
activity_uri = 'http://www.w3.org/ns/prov#Activity'
probe_result = probe_relationships(meta, activity_uri, limit=5)

print(f"Probing: {probe_result['label']}")
print(f"Outgoing relationships: {probe_result['outgoing_count']} total, showing {len(probe_result['outgoing'])}")
for rel in probe_result['outgoing'][:3]:
    print(f"  --{rel['pred_label']}--> {rel['obj_label']}")

print(f"\nIncoming relationships: {probe_result['incoming_count']} total, showing {len(probe_result['incoming'])}")
for rel in probe_result['incoming'][:3]:
    print(f"  <--{rel['pred_label']}-- {rel['subj_label']}")

# Test find_path
# Find path between two PROV classes
entity_uri = 'http://www.w3.org/ns/prov#Entity'
paths = find_path(meta, activity_uri, entity_uri, max_depth=2, limit=3)

print(f"\n\nPaths from Activity to Entity:")
if paths:
    for i, path in enumerate(paths, 1):
        print(f"Path {i}:")
        for step in path:
            direction_sym = '-->' if step['direction'] == 'out' else '<--'
            pred_label = meta.labels.get(step['predicate'], step['predicate'])
            print(f"  {direction_sym} {pred_label}")
else:
    print("  No paths found")


In [ ]:
# Test predicate_frequency
print("Top 10 predicates by frequency:")
freq_results = predicate_frequency(meta, limit=10)
for r in freq_results:
    print(f"  {r['count']:4d} uses - {r['label']}")

# Test filtering by predicate type
print("\nTop 5 object properties:")
obj_props = predicate_frequency(meta, limit=5, predicate_type='object')
for r in obj_props:
    print(f"  {r['count']:4d} uses - {r['label']}")

## Additional Exploration Functions

Functions discovered in `dialogs/inspect_tools.ipynb` for deeper ontology exploration.

In [ ]:
#| export
def ont_describe(ont: str, uri: str, name: str = 'desc', ns: dict = None) -> str:
    """Get all triples about a URI, store in namespace.
    
    Returns both triples where URI is subject and where it's object.
    
    Args:
        ont: Name of ontology variable in namespace
        uri: URI to describe
        name: Variable name for storing result
        ns: Namespace dict
        
    Returns:
        Summary string
    """
    if ns is None: ns = globals()
    o = ns[ont]
    u = URIRef(uri) if not isinstance(uri, URIRef) else uri
    
    # Get triples where URI is subject
    subj_triples = [(str(s), str(p), str(obj)) for s, p, obj in o.graph.triples((u, None, None))]
    
    # Get triples where URI is object
    obj_triples = [(str(s), str(p), str(obj)) for s, p, obj in o.graph.triples((None, None, u))]
    
    result = {
        'as_subject': subj_triples,
        'as_object': obj_triples
    }
    ns[name] = result
    return f"Stored {len(subj_triples)} + {len(obj_triples)} triples about '{uri}' into '{name}'"

In [ ]:
#| export
def ont_meta(ont: str, name: str = 'meta', ns: dict = None) -> str:
    """Extract ontology metadata (prefixes, annotation predicates, imports).
    
    Args:
        ont: Name of ontology variable in namespace
        name: Variable name for storing result
        ns: Namespace dict
        
    Returns:
        Summary string
    """
    if ns is None: ns = globals()
    o = ns[ont]
    
    prefixes = dict(o.graph.namespaces())
    ann_preds = set(str(p) for s, p, obj in o.graph.triples((None, None, None)) if isinstance(obj, Literal))
    imports = [str(obj) for s, p, obj in o.graph.triples((None, OWL.imports, None))]
    
    res = AttrDict(
        prefixes=prefixes,
        ann_preds=list(ann_preds)[:50],  # Limit to first 50
        imports=imports
    )
    ns[name] = res
    return f"Stored metadata into '{name}': {len(prefixes)} prefixes, {len(ann_preds)} annotation predicates, {len(imports)} imports"

In [ ]:
#| export
def ont_roots(ont: str, name: str = 'roots', ns: dict = None) -> str:
    """Find root classes (no declared superclass), store in namespace.
    
    Args:
        ont: Name of ontology variable in namespace
        name: Variable name for storing result
        ns: Namespace dict
        
    Returns:
        Summary string
    """
    if ns is None: ns = globals()
    o = ns[ont]
    
    has_super = set(o.supers.keys())
    roots = [str(c) for c in o.classes if str(c).startswith('http') and str(c) not in has_super]
    
    ns[name] = roots
    return f"Stored {len(roots)} root classes into '{name}'"

In [ ]:
#| export
def setup_ontology_context(path: str | Path, ns: dict, name: str = 'ont', dataset_meta=None) -> str:
    """Load ontology and create meta-graph for RLM use.
    
    This sets up both the Graph and GraphMeta in the namespace.
    
    NEW: Dataset integration - if dataset_meta provided, automatically mounts
    the ontology into the dataset as onto/<name> graph.
    
    Args:
        path: Path to ontology file
        ns: Namespace dict
        name: Base name for graph handle
        dataset_meta: Optional DatasetMeta for auto-mounting
        
    Returns:
        Summary string
    """
    # Load graph
    load_msg = load_ontology(path, ns, name=name)
    
    # Create meta-graph
    g = ns[name]
    meta = GraphMeta(g, name=name)
    ns[f"{name}_meta"] = meta
    
    # NEW: Auto-mount in dataset if provided
    if dataset_meta is not None:
        try:
            from rlm.dataset import mount_ontology
            mount_msg = mount_ontology(dataset_meta, ns, str(path), name)
            load_msg += f"\n{mount_msg}"
        except Exception as e:
            print(f"Warning: Failed to mount ontology in dataset: {e}")
    
    # FIX: Namespace helper functions by ontology name to avoid overwriting
    # This allows multiple ontologies to coexist
    from functools import partial
    
    # Bind existing functions
    ns[f'{name}_graph_stats'] = partial(graph_stats, meta)
    ns[f'{name}_search_by_label'] = partial(search_by_label, meta)
    ns[f'{name}_describe_entity'] = partial(describe_entity, meta)
    
    # NEW: Bind Stage 2 bounded view primitives
    ns[f'{name}_search_entity'] = partial(search_entity, meta)
    ns[f'{name}_probe_relationships'] = partial(probe_relationships, meta)
    ns[f'{name}_find_path'] = partial(find_path, meta)
    ns[f'{name}_predicate_frequency'] = partial(predicate_frequency, meta)
    
    # Also bind without prefix for single-ontology convenience
    # (will be overwritten if multiple ontologies loaded, but prefixed versions persist)
    ns['graph_stats'] = partial(graph_stats, meta)
    ns['search_by_label'] = partial(search_by_label, meta)
    ns['describe_entity'] = partial(describe_entity, meta)
    ns['search_entity'] = partial(search_entity, meta)
    ns['probe_relationships'] = partial(probe_relationships, meta)
    ns['find_path'] = partial(find_path, meta)
    ns['predicate_frequency'] = partial(predicate_frequency, meta)
    
    return f"{load_msg}\nCreated meta-graph '{name}_meta' with {len(meta.classes)} classes, {len(meta.properties)} properties"

## Ontology Sense Building

### What is a "Sense Document"?

When an LLM needs to work with an ontology, loading the entire graph into context is wasteful and may exceed limits. Instead, we build a **sense document** - a compact summary that captures:

- **Formalism**: Which OWL/RDFS/SKOS constructs are used
- **Metadata structure**: Which annotation properties exist (labels, descriptions, etc.)
- **Domain/scope**: What the ontology is about
- **Navigation hints**: How to effectively search and traverse

This approach was developed through experiments in `dialogs/inspect_tools.ipynb` exploring progressive disclosure patterns.

### Why Sense Building Matters

**Design Decision Response** (from ISSUE_ANALYSIS.md):
> *GraphMeta.labels only uses rdfs:label* - This is a limitation because different ontologies use different annotation properties:
> - `rdfs:label`, `skos:prefLabel`, `skos:altLabel` for labels
> - `rdfs:comment`, `skos:definition`, `dcterms:description` for descriptions  
> - `vann:preferredNamespacePrefix`, `owl:versionInfo` for metadata

Rather than hardcode support for all possible properties, `build_sense()` **detects which annotation properties this specific ontology uses**, enabling intelligent search.

### References

- [Widoco Metadata Guide](https://github.com/dgarijo/Widoco/blob/master/doc/metadataGuide/guide.md) - Recommended ontology metadata properties
- [Anthropic: Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) - Orchestrator-workers pattern
- [Anthropic: Progressive Disclosure](https://www.anthropic.com/engineering/effective-context-engineering-for-ai-agents) - Context engineering strategy

### Implementation Pattern

The sense-building workflow (not agentic):
1. **Metadata collection** - Extract prefixes, detect annotation predicates, find ontology-level metadata
2. **Structural exploration** - Build hierarchy, property signatures, detect OWL axioms
3. **LLM synthesis** - One LLM call to identify domain, patterns, navigation hints
4. **Structured storage** - Store as retrievable AttrDict in REPL namespace

In [ ]:
#| export
def build_sense(path: str, name: str = 'sense', ns: dict = None) -> str:
    """Build ontology sense document using workflow + LLM synthesis.
    
    Detects annotation properties per Widoco metadata guide:
    - Label properties: rdfs:label, skos:prefLabel, skos:altLabel, dcterms:title
    - Description properties: rdfs:comment, skos:definition, dcterms:description
    - Ontology metadata: vann:preferredNamespacePrefix, owl:versionInfo, etc.
    
    This function:
    1. Loads ontology and extracts metadata/roots programmatically
    2. Detects which annotation properties are actually used
    3. Builds hierarchy (2 levels), property info, characteristics
    4. Makes one LLM call to synthesize domain/scope/patterns/hints
    5. Returns structured AttrDict stored in namespace
    
    Args:
        path: Path to ontology file
        name: Variable name for sense document (default: 'sense')
        ns: Namespace dict
        
    Returns:
        Summary string
    """
    if ns is None: ns = {}
    
    # Derive ontology name from sense name
    ont_name = name.replace('_sense', '').replace('sense', 'ont')
    
    # Setup ontology context (loads graph + creates GraphMeta)
    setup_ontology_context(path, ns, name=ont_name)
    
    # Get metadata and roots
    ont_meta(f'{ont_name}_meta', name=f'{ont_name}_metadata', ns=ns)
    ont_roots(f'{ont_name}_meta', name=f'{ont_name}_roots', ns=ns)
    
    # Get references
    meta_obj = ns[f'{ont_name}_meta']
    metadata = ns[f'{ont_name}_metadata']
    roots = ns[f'{ont_name}_roots']
    g = meta_obj.graph
    
    # Detect ontology-level metadata (Widoco guide properties)
    ont_uri = None
    for s in g.subjects(RDF.type, OWL.Ontology):
        ont_uri = str(s)
        break
    
    ont_metadata = {}
    if ont_uri:
        ont_ref = URIRef(ont_uri)
        ont_metadata['uri'] = ont_uri
        
        # Title/label
        for title_prop in [DCTERMS.title, DC.title, RDFS.label]:
            titles = list(g.objects(ont_ref, title_prop))
            if titles:
                ont_metadata['title'] = str(titles[0])
                break
        
        # Description
        for desc_prop in [DCTERMS.description, DC.description, RDFS.comment]:
            descs = list(g.objects(ont_ref, desc_prop))
            if descs:
                ont_metadata['description'] = str(descs[0])
                break
        
        # Version info
        versions = list(g.objects(ont_ref, OWL.versionInfo))
        if versions:
            ont_metadata['version'] = str(versions[0])
        
        # Preferred namespace prefix
        prefixes = list(g.objects(ont_ref, VANN.preferredNamespacePrefix))
        if prefixes:
            ont_metadata['preferred_prefix'] = str(prefixes[0])
        
        # Preferred namespace URI
        ns_uris = list(g.objects(ont_ref, VANN.preferredNamespaceUri))
        if ns_uris:
            ont_metadata['preferred_namespace'] = str(ns_uris[0])
    
    # Detect which annotation properties are actually used (per Widoco guide)
    label_props = []
    desc_props = []
    
    # Check label properties
    for label_prop in [RDFS.label, SKOS.prefLabel, SKOS.altLabel, DCTERMS.title, DC.title]:
        if list(g.triples((None, label_prop, None))):
            label_props.append(str(label_prop))
    
    # Check description properties
    for desc_prop in [RDFS.comment, SKOS.definition, SKOS.note, SKOS.example, 
                      DCTERMS.description, DC.description]:
        if list(g.triples((None, desc_prop, None))):
            desc_props.append(str(desc_prop))
    
    # Build hierarchy (2 levels deep from roots)
    hier = {}
    for r in roots[:10]:  # Limit to first 10 roots
        lbl = meta_obj.labels.get(r, r)
        children = meta_obj.subs.get(r, [])
        hier[lbl] = {
            meta_obj.labels.get(c, c): [
                meta_obj.labels.get(gc, gc) 
                for gc in meta_obj.subs.get(c, [])[:5]
            ] 
            for c in children[:10]
        }
    
    # Extract top properties with domains/ranges
    top_props = []
    for p in meta_obj.properties[:20]:
        if p.startswith('http'):
            prop_label = meta_obj.labels.get(p, p)
            dom_uri = meta_obj.doms.get(p, '')
            rng_uri = meta_obj.rngs.get(p, '')
            dom_label = meta_obj.labels.get(dom_uri, dom_uri) if dom_uri else ''
            rng_label = meta_obj.labels.get(rng_uri, rng_uri) if rng_uri else ''
            top_props.append((prop_label, dom_label, rng_label))
    
    # Detect property characteristics (OWL axioms)
    prop_chars = {}
    for p in meta_obj.properties[:50]:
        if p.startswith('http'):
            chars = []
            p_uri = URIRef(p)
            
            # Check for transitive
            if list(g.triples((p_uri, RDF.type, OWL.TransitiveProperty))):
                chars.append('transitive')
            
            # Check for symmetric
            if list(g.triples((p_uri, RDF.type, OWL.SymmetricProperty))):
                chars.append('symmetric')
            
            # Check for functional
            if list(g.triples((p_uri, RDF.type, OWL.FunctionalProperty))):
                chars.append('functional')
            
            # Check for inverse functional
            if list(g.triples((p_uri, RDF.type, OWL.InverseFunctionalProperty))):
                chars.append('inverse_functional')
            
            # Check for inverse
            if list(g.triples((p_uri, OWL.inverseOf, None))):
                chars.append('has_inverse')
            
            if chars:
                prop_chars[p] = chars
    
    # Detect OWL constructs usage
    owl_constructs = {
        'restrictions': len(list(g.subjects(RDF.type, OWL.Restriction))),
        'unions': len(list(g.subjects(OWL.unionOf, None))),
        'intersections': len(list(g.subjects(OWL.intersectionOf, None))),
        'disjointness': len(list(g.triples((None, OWL.disjointWith, None)))),
        'equivalence': len(list(g.triples((None, OWL.equivalentClass, None))))
    }
    
    # Get URI pattern samples
    uri_sample = [c for c in meta_obj.classes[:5] if c.startswith('http')]
    uri_pattern = uri_sample[0].rsplit('/', 1)[0] if uri_sample else ''
    
    # Build prompt for LLM synthesis
    prompt = f"""Analyze this ontology and provide a sense document:

**Ontology Metadata:**
{ont_metadata}

**Stats:** {len(meta_obj.classes)} classes, {len(meta_obj.properties)} properties, {len(meta_obj.labels)} labels

**Annotation Properties Detected:**
- Label properties: {label_props}
- Description properties: {desc_props}

**Structure:**
- Prefixes: {list(metadata.prefixes.keys())[:10]}
- Root classes: {[meta_obj.labels.get(r, r) for r in roots[:10]]}
- Hierarchy (2 levels): {hier}

**Properties:**
- Top properties (label, domain, range): {top_props[:10]}
- Property characteristics: {prop_chars}

**OWL Constructs Usage:**
{owl_constructs}

**URI Pattern:** {uri_pattern}
- Sample URIs: {uri_sample[:3]}

Provide a concise sense document with:
1) Domain/scope - what is this ontology about?
2) Key branches - main conceptual areas in the hierarchy
3) Important properties - key relationships to know
4) Detected patterns - reification, measurement patterns, part-whole relationships, restriction patterns, etc.
5) SPARQL navigation hints - how to effectively query this ontology given the annotation properties available"""
    
    # Use llm_query from rlm.core
    from rlm.core import llm_query
    summary = llm_query(prompt, ns=ns, name='_sense_summary')
    
    # Build structured sense document
    sense_doc = AttrDict(
        ont=ont_name,
        ont_metadata=ont_metadata,
        stats={'cls': len(meta_obj.classes), 'props': len(meta_obj.properties), 'lbls': len(meta_obj.labels)},
        prefixes=metadata.prefixes,
        label_properties=label_props,  # NEW: detected label properties
        description_properties=desc_props,  # NEW: detected description properties
        ann_preds=metadata.ann_preds,
        roots=roots,
        hier=hier,
        top_props=top_props,
        prop_chars=prop_chars,
        owl_constructs=owl_constructs,  # NEW: OWL construct usage
        uri_pattern=uri_pattern,
        summary=ns['_sense_summary']
    )
    
    ns[name] = sense_doc
    return f"Built sense document into '{name}': {len(label_props)} label properties, {len(desc_props)} description properties, {len(hier)} root branches"

In [ ]:
#| eval: false
# Test build_sense with PROV ontology
# Note: Requires API key, marked eval:false to avoid CI failures

test_ns = {}
result = build_sense('ontology/prov.ttl', name='prov_sense', ns=test_ns)
print(result)
print()

# Inspect the sense document
sense = test_ns['prov_sense']
print(f"Ontology: {sense.ont}")
print(f"Ontology Metadata: {sense.ont_metadata}")
print(f"Stats: {sense.stats}")
print()

# NEW: Show detected annotation properties
print(f"Label properties detected: {sense.label_properties}")
print(f"Description properties detected: {sense.description_properties}")
print()

print(f"Roots: {sense.roots}")
print(f"Root branches: {list(sense.hier.keys())}")
print(f"Top properties (first 3): {sense.top_props[:3]}")
print(f"Property characteristics: {sense.prop_chars}")
print(f"OWL constructs: {sense.owl_constructs}")
print(f"URI pattern: {sense.uri_pattern}")
print()
print("LLM Summary:")
print(sense.summary)

## Integration with RLM

Helper to setup ontology context for `rlm_run()`.

In [ ]:
# NOTE: setup_ontology_context() is defined above in cell-27
# This cell previously contained a duplicate definition

In [ ]:
# Test setup for RLM
test_ns = {}
result = setup_ontology_context('ontology/prov.ttl', test_ns, name='prov')
print(result)
print()
print("Namespace contains:")
for k in test_ns.keys():
    print(f"  {k}: {type(test_ns[k]).__name__}")

In [ ]:
# Test new exploration functions
# Reuse the test_ns from previous cell with loaded prov ontology
# Note: prov_meta is a GraphMeta object in test_ns

# Test that new indexes work
meta = test_ns['prov_meta']
assert len(meta.by_label) > 0  # inverted label index
assert len(meta.subs) > 0 or len(meta.supers) > 0  # class hierarchy
print(f"✓ New GraphMeta indexes work: by_label has {len(meta.by_label)} entries")

# Test ont_describe (need to pass GraphMeta object as namespace entry)
result = ont_describe('prov_meta', 'http://www.w3.org/ns/prov#Activity', name='activity_desc', ns=test_ns)
assert 'activity_desc' in test_ns
print(f"✓ ont_describe works: {result}")

# Test ont_meta  
result = ont_meta('prov_meta', name='prov_metadata', ns=test_ns)
assert 'prov_metadata' in test_ns
print(f"✓ ont_meta works: {result}")

# Test ont_roots
result = ont_roots('prov_meta', name='prov_roots', ns=test_ns)
assert 'prov_roots' in test_ns
print(f"✓ ont_roots works: {result}")

## Test with RLM

Now let's test asking a question about the PROV ontology using `rlm_run()`.

In [ ]:
#| eval: false
from rlm.core import rlm_run

# Setup namespace with PROV ontology
ns = {}
setup_ontology_context('ontology/prov.ttl', ns, name='prov')

# Ask a question
# The context is the GraphMeta summary, not the full graph
context = ns['prov_meta'].summary()

answer, iterations, ns = rlm_run(
    "What is the Activity class in the PROV ontology?",
    context,
    ns=ns,
    max_iters=3
)

print(f"Answer: {answer}")
print(f"Iterations: {len(iterations)}")